# Лабораторная 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from jinja2.utils import concat


## I. Подготовка данных и базовые операции с NumPy

### 1. Загрузка и подготовка данных:

#### Загрузите набор данных о прокате велосипедов в Сеуле и исследуйте его структуру.

In [183]:
# загрузка данных
data = np.genfromtxt('SeoulBikeData.csv', delimiter=',', names=True, dtype=None)


In [184]:
# первые 5 строк
data[:5]


array([('01/12/2017', 254, 0, -5.2, 37, 2.2, 2000, -17.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes'),
       ('01/12/2017', 204, 1, -5.5, 38, 0.8, 2000, -17.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes'),
       ('01/12/2017', 173, 2, -6. , 39, 1. , 2000, -17.7, 0., 0., 0., 'Winter', 'No Holiday', 'Yes'),
       ('01/12/2017', 107, 3, -6.2, 40, 0.9, 2000, -17.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes'),
       ('01/12/2017',  78, 4, -6. , 36, 2.3, 2000, -18.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes')],
      dtype=[('Date', '<U10'), ('Rented_Bike_Count', '<i8'), ('Hour', '<i8'), ('Temperature°C', '<f8'), ('Humidity', '<i8'), ('Wind_speed_ms', '<f8'), ('Visibility_10m', '<i8'), ('Dew_point_temperature°C', '<f8'), ('Solar_Radiation_MJm2', '<f8'), ('Rainfallmm', '<f8'), ('Snowfall_cm', '<f8'), ('Seasons', '<U6'), ('Holiday', '<U10'), ('Functioning_Day', '<U3')])

In [185]:
# размер данных
data_length = data.shape[0]
data_length


8760

In [186]:
# анализ каждого столбца
for name in data.dtype.names:
    unique_values = np.unique(data[name])

    print(name)
    print('Тип:', unique_values.dtype)

    if len(unique_values) < 5:
        print('Уникальные значения:', unique_values)
    else:
        print('Первые 5 уникальных значений:', unique_values[:5], 'Всего:', len(unique_values))

    if np.issubdtype(unique_values.dtype, np.number):
        print('Минимум:', np.min(unique_values), 'Максимум:', np.max(unique_values))

    print()


Date
Тип: <U10
Первые 5 уникальных значений: ['01/01/2018' '01/02/2018' '01/03/2018' '01/04/2018' '01/05/2018'] Всего: 365

Rented_Bike_Count
Тип: int64
Первые 5 уникальных значений: [0 2 3 4 5] Всего: 2166
Минимум: 0 Максимум: 3556

Hour
Тип: int64
Первые 5 уникальных значений: [0 1 2 3 4] Всего: 24
Минимум: 0 Максимум: 23

Temperature°C
Тип: float64
Первые 5 уникальных значений: [-17.8 -17.5 -17.4 -16.9 -16.5] Всего: 546
Минимум: -17.8 Максимум: 39.4

Humidity
Тип: int64
Первые 5 уникальных значений: [ 0 10 11 12 13] Всего: 90
Минимум: 0 Максимум: 98

Wind_speed_ms
Тип: float64
Первые 5 уникальных значений: [0.  0.1 0.2 0.3 0.4] Всего: 65
Минимум: 0.0 Максимум: 7.4

Visibility_10m
Тип: int64
Первые 5 уникальных значений: [27 33 34 38 53] Всего: 1789
Минимум: 27 Максимум: 2000

Dew_point_temperature°C
Тип: float64
Первые 5 уникальных значений: [-30.6 -30.5 -29.8 -29.7 -29.6] Всего: 556
Минимум: -30.6 Максимум: 27.2

Solar_Radiation_MJm2
Тип: float64
Первые 5 уникальных значений: [0.  

#### Выполните предварительную обработку данных (например, удалите пропущенные значения или преобразуйте категориальные переменные).

In [187]:
# проверка на пропущенные значения
for name in data.dtype.names:
    if np.issubdtype(data[name].dtype, np.number):
        print(name, np.isnan(data[name]).sum())
    else:
        print(name, np.char.equal(data[name], '').sum())


Date 0
Rented_Bike_Count 0
Hour 0
Temperature°C 0
Humidity 0
Wind_speed_ms 0
Visibility_10m 0
Dew_point_temperature°C 0
Solar_Radiation_MJm2 0
Rainfallmm 0
Snowfall_cm 0
Seasons 0
Holiday 0
Functioning_Day 0


In [188]:
# преобразование даты из строкового формата Д/М/Г в числовой формат
date = data['Date']

date = np.array(
    [np.datetime64(
        f'{d.split('/')[2]}-{d.split('/')[1]}-{d.split('/')[0]}')
        for d in date])

date_numeric = date.astype('int')
date_numeric = date_numeric - np.min(date_numeric)

(date, date_numeric)


(array(['2017-12-01', '2017-12-01', '2017-12-01', ..., '2018-11-30',
        '2018-11-30', '2018-11-30'], shape=(8760,), dtype='datetime64[D]'),
 array([  0,   0,   0, ..., 364, 364, 364], shape=(8760,)))

In [189]:
# преобразование категориальных переменных
season = data['Seasons']
season_numeric = np.array([0 if s == 'Winter' else 1 if s == 'Spring' else 2 if s == 'Summer' else 3 for s in season])

holiday = data['Holiday']
holiday_numeric = np.array([1 if h == 'Holiday' else 0 for h in holiday])

functioning_day = data['Functioning_Day']
functioning_day_numeric = np.array([1 if f == 'Yes' else 0 for f in functioning_day])

(season, season_numeric, holiday, holiday_numeric, functioning_day, functioning_day_numeric)


(array(['Winter', 'Winter', 'Winter', ..., 'Autumn', 'Autumn', 'Autumn'],
       shape=(8760,), dtype='<U6'),
 array([0, 0, 0, ..., 3, 3, 3], shape=(8760,)),
 array(['No Holiday', 'No Holiday', 'No Holiday', ..., 'No Holiday',
        'No Holiday', 'No Holiday'], shape=(8760,), dtype='<U10'),
 array([0, 0, 0, ..., 0, 0, 0], shape=(8760,)),
 array(['Yes', 'Yes', 'Yes', ..., 'Yes', 'Yes', 'Yes'],
       shape=(8760,), dtype='<U3'),
 array([1, 1, 1, ..., 1, 1, 1], shape=(8760,)))

In [190]:
# новая матрица числовых данных
columns = [
    ("Date", date.dtype, date),
    *[
        (name, np.float64, data[name]) if np.issubdtype(data[name].dtype, np.number)
        else (name, data[name].dtype, data[name])
        for name in data.dtype.names[1:]
    ],
    ("Date_numeric", np.float64, date_numeric),
    ("Season_numeric", np.float64, season_numeric),
    ("Holiday_numeric", np.float64, holiday_numeric),
    ("Functioning_day_numeric", np.float64, functioning_day_numeric),
]

dtype = [(name, dtype) for name, dtype, _ in columns]
data = np.array(list(zip(*[values for _, _, values in columns])), dtype=dtype)

data[:5]


array([('2017-12-01', 254., 0., -5.2, 37., 2.2, 2000., -17.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 204., 1., -5.5, 38., 0.8, 2000., -17.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 173., 2., -6. , 39., 1. , 2000., -17.7, 0., 0., 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 107., 3., -6.2, 40., 0.9, 2000., -17.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01',  78., 4., -6. , 36., 2.3, 2000., -18.6, 0., 0., 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.)],
      dtype=[('Date', '<M8[D]'), ('Rented_Bike_Count', '<f8'), ('Hour', '<f8'), ('Temperature°C', '<f8'), ('Humidity', '<f8'), ('Wind_speed_ms', '<f8'), ('Visibility_10m', '<f8'), ('Dew_point_temperature°C', '<f8'), ('Solar_Radiation_MJm2', '<f8'), ('Rainfallmm', '<f8'), ('Snowfall_cm', '<f8'), ('Seasons', '<U6'), ('Holiday', '<U10'), ('Functioning_Day', '<U3'), ('Date_numeric', '<f8'),

### 2. Основные операции с NumPy:


#### Создайте векторы и матрицы с использованием np.array(), соответствующие данным из набора. Это могут быть временные ряды или другие важные метрики.

#### Используйте матрицы и векторы для представления различных параметров, таких как количество прокачанных велосипедов, температура, влажность и другие переменные.

##### Векторы:

In [191]:
date = data['Date']
date_numeric = data['Date_numeric']
rented_bike_count = data['Rented_Bike_Count']
hour = data['Hour']
temperature = data['Temperature°C']
humidity = data['Humidity']
wind_speed = data['Wind_speed_ms']
visibility = data['Visibility_10m']
dew_point_temperature = data['Dew_point_temperature°C']
solar_radiation = data['Solar_Radiation_MJm2']
rainfall = data['Rainfallmm']
snowfall = data['Snowfall_cm']
season = data['Seasons']
season_numeric = data['Season_numeric']
holiday = data['Holiday']
holiday_numeric = data['Holiday_numeric']
functioning_day = data['Functioning_Day']
functioning_day_numeric = data['Functioning_day_numeric']

(date, date_numeric, rented_bike_count, hour, temperature, humidity, wind_speed, visibility, dew_point_temperature,
 solar_radiation, rainfall, snowfall, season, season_numeric, holiday, holiday_numeric, functioning_day,
 functioning_day_numeric)


(array(['2017-12-01', '2017-12-01', '2017-12-01', ..., '2018-11-30',
        '2018-11-30', '2018-11-30'], shape=(8760,), dtype='datetime64[D]'),
 array([  0.,   0.,   0., ..., 364., 364., 364.], shape=(8760,)),
 array([254., 204., 173., ..., 694., 712., 584.], shape=(8760,)),
 array([ 0.,  1.,  2., ..., 21., 22., 23.], shape=(8760,)),
 array([-5.2, -5.5, -6. , ...,  2.6,  2.1,  1.9], shape=(8760,)),
 array([37., 38., 39., ..., 39., 41., 43.], shape=(8760,)),
 array([2.2, 0.8, 1. , ..., 0.3, 1. , 1.3], shape=(8760,)),
 array([2000., 2000., 2000., ..., 1968., 1859., 1909.], shape=(8760,)),
 array([-17.6, -17.6, -17.7, ...,  -9.9,  -9.8,  -9.3], shape=(8760,)),
 array([0., 0., 0., ..., 0., 0., 0.], shape=(8760,)),
 array([0., 0., 0., ..., 0., 0., 0.], shape=(8760,)),
 array([0., 0., 0., ..., 0., 0., 0.], shape=(8760,)),
 array(['Winter', 'Winter', 'Winter', ..., 'Autumn', 'Autumn', 'Autumn'],
       shape=(8760,), dtype='<U6'),
 array([0., 0., 0., ..., 3., 3., 3.], shape=(8760,)),
 array(

##### Матрицы:

In [192]:
# матрица погодных условий
weather_matrix = np.array([
    temperature,
    humidity,
    wind_speed,
    visibility,
    dew_point_temperature,
    solar_radiation,
    rainfall,
    snowfall,
]).T
weather_matrix


array([[-5.2, 37. ,  2.2, ...,  0. ,  0. ,  0. ],
       [-5.5, 38. ,  0.8, ...,  0. ,  0. ,  0. ],
       [-6. , 39. ,  1. , ...,  0. ,  0. ,  0. ],
       ...,
       [ 2.6, 39. ,  0.3, ...,  0. ,  0. ,  0. ],
       [ 2.1, 41. ,  1. , ...,  0. ,  0. ,  0. ],
       [ 1.9, 43. ,  1.3, ...,  0. ,  0. ,  0. ]], shape=(8760, 8))

In [193]:
# матрица дат
date_matrix = np.array([
    date,
    date_numeric,
    season,
    season_numeric,
    holiday,
    holiday_numeric,
    functioning_day,
    functioning_day_numeric,
]).T
date_matrix


array([[datetime.date(2017, 12, 1), 0.0, 'Winter', ..., 0.0, 'Yes', 1.0],
       [datetime.date(2017, 12, 1), 0.0, 'Winter', ..., 0.0, 'Yes', 1.0],
       [datetime.date(2017, 12, 1), 0.0, 'Winter', ..., 0.0, 'Yes', 1.0],
       ...,
       [datetime.date(2018, 11, 30), 364.0, 'Autumn', ..., 0.0, 'Yes',
        1.0],
       [datetime.date(2018, 11, 30), 364.0, 'Autumn', ..., 0.0, 'Yes',
        1.0],
       [datetime.date(2018, 11, 30), 364.0, 'Autumn', ..., 0.0, 'Yes',
        1.0]], shape=(8760, 8), dtype=object)

### 3. Индексация и срезы:

#### Извлеките подматрицы данных (например, данные по отдельным дням или месяцам).


In [195]:
# данные за первые 5 дней

first_five_days_mask = data['Date'] < np.datetime64('2017-12-06')
first_five_days_data = data[first_five_days_mask]

first_five_days_data


array([('2017-12-01', 254.,  0., -5.2, 37., 2.2, 2000., -17.6, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 204.,  1., -5.5, 38., 0.8, 2000., -17.6, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 173.,  2., -6. , 39., 1. , 2000., -17.7, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 107.,  3., -6.2, 40., 0.9, 2000., -17.6, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01',  78.,  4., -6. , 36., 2.3, 2000., -18.6, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 100.,  5., -6.4, 37., 1.5, 2000., -18.7, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 181.,  6., -6.6, 35., 1.3, 2000., -19.5, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0., 1.),
       ('2017-12-01', 460.,  7., -7.4, 38., 0.9, 2000., -19.3, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 0., 0., 0

In [196]:
# данные за первые 5 дней, кроме первого

first_five_days_mask_without_first = (data['Date'] < np.datetime64('2017-12-06')) & (data['Date'] > np.datetime64('2017-12-01'))
first_five_days_data_without_first = data[first_five_days_mask_without_first]

first_five_days_data_without_first


array([('2017-12-02', 328.,  0., -1.8, 87., 1.1,  994.,  -3.6, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02', 308.,  1., -2.2, 86., 0.6,  990.,  -4.2, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02', 262.,  2., -2.9, 86., 1.5, 1256.,  -4.9, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02', 167.,  3., -3.5, 81., 2.2, 1221.,  -6.2, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02',  89.,  4., -3.8, 79., 2. , 1167.,  -6.9, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02',  79.,  5., -4. , 79., 1.5, 1202.,  -7.1, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02',  70.,  6., -4.3, 82., 2.1, 1178.,  -6.9, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0., 1.),
       ('2017-12-02', 146.,  7., -4.4, 81., 2.5, 1276.,  -7.1, 0.  , 0. , 0., 'Winter', 'No Holiday', 'Yes', 1., 0., 0

In [197]:
# данные за первый час каждого дня июня 2018 года

first_hour_of_june_2018_mask = (data['Date'] >= np.datetime64('2018-06-01')) & (data['Date'] < np.datetime64('2018-07-01')) & (data['Hour'] == 0)
first_hour_of_june_2018_data = data[first_hour_of_june_2018_mask]

first_hour_of_june_2018_data


array([('2018-06-01',  963., 0., 20.1, 63., 1.1, 1698., 12.8, 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 182., 2., 0., 1.),
       ('2018-06-02', 1165., 0., 20.9, 53., 1.4, 1717., 10.9, 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 183., 2., 0., 1.),
       ('2018-06-03', 1248., 0., 19.7, 55., 2. , 1986., 10.3, 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 184., 2., 0., 1.),
       ('2018-06-04',  938., 0., 21.4, 45., 1.4, 1964.,  8.9, 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 185., 2., 0., 1.),
       ('2018-06-05',  927., 0., 20.5, 71., 2.8, 1204., 15. , 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 186., 2., 0., 1.),
       ('2018-06-06', 1268., 0., 21.2, 60., 0.9, 1789., 13.1, 0., 0., 0., 'Summer', 'Holiday', 'Yes', 187., 2., 1., 1.),
       ('2018-06-07',  978., 0., 19.6, 82., 1.7,  628., 16.4, 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 188., 2., 0., 1.),
       ('2018-06-08',  926., 0., 19.1, 82., 2.2,  874., 15.9, 0., 0., 0., 'Summer', 'No Holiday', 'Yes', 189., 2., 0., 1.),
       ('20